# Import helper functions

In [ ]:
%run ../0.shared_notebooks/0_helper_functions.ipynb

# Set constants

In [ ]:
IN_DB="../IN/stealer.sqlite3"
OUT_DIR="../NORMALIZED"

# Open DB

In [ ]:
# Open database
conn = sqlite3.connect(IN_DB)
in_cur = conn.cursor()
out_cur = conn.cursor()

In [ ]:
conn

## Add NML_hash and masked password

In [ ]:
in_cur.execute('select count(*) from "stealer" where nml_hash is null collate nocase')
todo = in_cur.fetchone()[0]
if todo :
    count = 0
    done = -1
    # Progress bar
    widgets=[
        progressbar.SimpleProgress(), 
        ' (', progressbar.Percentage(format='%(percentage)3.2f%%'), ') ',
        progressbar.Bar(),
        progressbar.ETA(),
    ]
    bar = progressbar.ProgressBar(max_value=todo, widgets=widgets,enable_colors=False)
    bar.update(1)
    while done != 0:
        done = 0
        in_cur.execute('select rowid, user, passwd from stealer where nml_hash is null limit 10000 collate nocase')
        for r in in_cur:
            out_cur.execute("""
                     UPDATE 'stealer' set nml_hash = ?, masked_passwd = ?, has_name = false, has_dob = false, has_addr = false, 
                         has_phone = false, has_cc = false, has_bankacc = false, has_ssn = false, has_ip = false where rowid = ?
                """, (
                    nml_hash(r[1],r[2]), mask_password(r[2]), r[0]
                )
            )
            count=count+1
            done = done + 1
            if count % 1000 == 0 :
                bar.update(count)
        if count % 1_000_000 == 0 :
            conn.commit()
    conn.commit()
else:
    print("Nothng to do here")

In [ ]:
# Remove database lock (if previous step failed)
conn.commit()

# Remove NML hash where user <> email

In [ ]:
%run ../0.shared_notebooks/0_helper_functions.ipynb

In [ ]:
conn=sqlite3.connect("../LEAK_DB/DIVD-2024-00022-set1.sqlite3")
in_cur=conn.cursor()
out_cur=conn.cursor()

In [ ]:
# Get non-email addresses
done = 0
total = 0
this_run = -1
last_rowid = 1_267_816_826
while this_run != 0 :
    if last_rowid:
        in_cur.execute("SELECT rowid, user FROM entity WHERE nml_hash IS NOT NULL and rowid > ? ORDER BY rowid LIMIT 10000;", (last_rowid, ) )
    else:
        in_cur.execute("SELECT rowid, user FROM entity WHERE nml_hash IS NOT NULL ORDER BY rowid LIMIT 50000;")
    not_email = []
    this_run=0
    for r in in_cur :
        this_run=this_run+1
        if is_email(r[1]):
            pass
        else:
            not_email.append(r[0])
        total=total+1
        last_rowid = r[0]
    for e in not_email :
        out_cur.execute("UPDATE entity SET nml_hash = NULL, user_apex = NULL WHERE rowid = ?", ( e, ) )
        done=done+1
    print(f"{total:,} / {done:,}  ({last_rowid})", end="\r")
    conn.commit

In [ ]:
last_rowid-10_000

In [ ]:
conn.commit()